**Connect google drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Jul 30 17:10:08 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': Operation not supported
'Colab Notebooks'   yolov3		 yolov3_training_1000.weights
 img		    yolov3_testing.cfg	 yolov3_training_last.weights


**1) Clone the Darknet**



In [ ]:
#for cloning darknet because we are using darknet open source for training our YOLOV3 model
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

[Errno 20] Not a directory: 'darknet'
/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
     

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: No such file or directory


In [ ]:
#creating object file for all the classes here it will be only one class(Nilgai).
!echo "nilgai" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

mkdir: cannot create directory ‘data/obj’: File exists


In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2020-07-30 18:38:05--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.1’

darknet53.conv.74.1 100%[===================>] 154.96M   481KB/s    in 7m 42s  

2020-07-30 18:45:48 (344 KB/s) - ‘darknet53.conv.74.1’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/images/Nilgai_1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/images/Nilgai_1.jpg  
  inflating: data/obj/images/Nilgai_1.txt  
  inflating: data/obj/images/Nilgai_10.jpg  
  inflating: data/obj/images/Nilgai_10.txt  
  inflating: data/obj/images/Nilgai_100.jpg  
  inflating: data/obj/images/Nilgai_100.txt  
  inflating: data/obj/images/Nilgai_1000.jpg  
  inflating: data/obj/images/Nilgai_1000.txt  
  inflating: data/obj/images/Nilgai_1001.jpg  
  inflating: data/obj/images/Nilgai_1001.txt  
  inflating: data/obj/images/Nilgai_1002.jpg  
  inflating: data/obj/images/Nilgai_1002.txt  
  inflating: data/obj/images/Nilgai_1003.jpg  
  inflating: data/obj/images/Nilgai_1003.txt  
  inflating: data/obj/images/Nilgai_1004.jpg  
  inflating: data/obj/images/Nilgai_1004.txt  
  inflating: data/obj/images/Nilgai_1005.jpg  
  inflating: data/obj/images/Nilgai_1005.txt  
  inflating: data/obj/images/Nilgai_1006.jpg  
  in

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/images/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

Streaming output truncated to the last 5000 lines.
236 data/obj/images/Nilgai_141.txt
0 0.399510 0.686275 0.661765 0.612745
0 0.577614 0.553922 0.844771 0.892157

['0', '0.577614', '0.553922', '0.844771', '0.892157']
237 data/obj/images/Nilgai_41.txt
0 0.577614 0.553922 0.844771 0.892157
0 0.443636 0.540761 0.341818 0.527174

['0', '0.443636', '0.540761', '0.341818', '0.527174']
238 data/obj/images/Nilgai_379.txt
0 0.443636 0.540761 0.341818 0.527174
0 0.391002 0.477273 0.364196 0.874545
['0', '0.391002', '0.477273', '0.364196', '0.874545']
239 data/obj/images/Nilgai_1019.txt
0 0.391002 0.477273 0.364196 0.874545
0 0.440075 0.497354 0.700375 0.878307

['0', '0.440075', '0.497354', '0.700375', '0.878307']
240 data/obj/images/Nilgai_406.txt
0 0.440075 0.497354 0.700375 0.878307
0 0.537855 0.414545 0.511702 0.701818
['0', '0.537855', '0.414545', '0.511702', '0.701818']
241 data/obj/images/Nilgai_1000.txt
0 0.537855 0.414545 0.511702 0.701818
0 0.522876 0.501225 0.493464 0.992647

['0', '0

In [ ]:
#save name of all the images in imga_list
import glob
images_list = glob.glob("data/obj/images/*.jpg")
print(images_list)

['data/obj/images/Nilgai_124.jpg', 'data/obj/images/Nilgai_308.jpg', 'data/obj/images/Nilgai_711.jpg', 'data/obj/images/Nilgai_670.jpg', 'data/obj/images/Nilgai_363.jpg', 'data/obj/images/Nilgai_164.jpg', 'data/obj/images/Nilgai_287.jpg', 'data/obj/images/Nilgai_784.jpg', 'data/obj/images/Nilgai_125.jpg', 'data/obj/images/Nilgai_474.jpg', 'data/obj/images/Nilgai_220.jpg', 'data/obj/images/Nilgai_968.jpg', 'data/obj/images/Nilgai_197.jpg', 'data/obj/images/Nilgai_668.jpg', 'data/obj/images/Nilgai_956.jpg', 'data/obj/images/Nilgai_976.jpg', 'data/obj/images/Nilgai_866.jpg', 'data/obj/images/Nilgai_251.jpg', 'data/obj/images/Nilgai_298.jpg', 'data/obj/images/Nilgai_77.jpg', 'data/obj/images/Nilgai_154.jpg', 'data/obj/images/Nilgai_254.jpg', 'data/obj/images/Nilgai_938.jpg', 'data/obj/images/Nilgai_396.jpg', 'data/obj/images/Nilgai_295.jpg', 'data/obj/images/Nilgai_706.jpg', 'data/obj/images/Nilgai_876.jpg', 'data/obj/images/Nilgai_948.jpg', 'data/obj/images/Nilgai_845.jpg', 'data/obj/imag

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 


**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.897300, GIOU: 0.896462), Class: 0.999093, Obj: 0.997707, No Obj: 0.005384, .5R: 1.000000, .75R: 1.000000, count: 3, class_loss = 0.000723, iou_loss = 0.045589, total_loss = 0.046312 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 94 Avg (IOU: 0.706913, GIOU: 0.686207), Class: 0.996582, Obj: 0.518753, No Obj: 0.000500, .5R: 1.000000, .75R: 0.333333, count: 3, class_loss = 0.408286, iou_loss = 0.192096, total_loss = 0.600382 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 106 Avg (IOU: 0.427230, GIOU: 0.396625), Class: 0.958688, Obj: 0.146718, No Obj: 0.000033, .5R: 0.000000, .75R: 0.000000, count: 1, class_loss = 0.196329, iou_loss = 0.337610, total_loss = 0.533939 
 total_bbox = 177426, rewritten_bbox = 0.013527 % 
v3 (mse loss, Normalizer: (iou: 0.75, cls: 1.00) Region 82 Avg (IOU: 0.857239, GIOU: 0.855048), Class: 0.998407, Obj: 0.901565, No Obj: